In [2]:
import logging

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

ModuleNotFoundError: No module named 'azureml.core'

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
ws = Workspace.from_config()

In [ ]:
# choose a name for the run history container in the workspace
experiment_name = 'automl-forecasting-energydemand'

experiment = Experiment(ws, experiment_name)

In [ ]:
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

In [ ]:
# Choose a name for your cluster.
amlcompute_cluster_name = 'olt-auto-ml' # Fill with compute name
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)

In [ ]:
dataset_name = 'energy-train'

# Get a dataset by name
energy_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [ ]:
energy_ds

In [ ]:
df = energy_ds.to_pandas_dataframe()

In [ ]:
df

In [ ]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters
forecasting_parameters = ForecastingParameters(
    time_column_name='Date',
    forecast_horizon=48
)

In [ ]:
automl_config = AutoMLConfig(task='forecasting',
                             primary_metric='r2_score',
                             training_data=energy_ds,
                             label_column_name='Load',
                             experiment_timeout_minutes=15,
                             compute_target=compute_target,
                             enable_voting_ensemble=True,
                             enable_early_stopping=True,
                             n_cross_validations=5,
                             forecasting_parameters=forecasting_parameters)

In [ ]:
remote_run = experiment.submit(automl_config, show_output=False)
remote_run

In [ ]:
remote_run.wait_for_completion()
best_run, fitted_model = remote_run.get_output()
fitted_model.steps